In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Adithya\\Desktop\\chest_cancer_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Adithya\\Desktop\\chest_cancer_classification'

In [5]:
import os
import dagshub
import mlflow

# Set the MLflow tracking URI
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/adithyab22/chest_cancer_classification.mlflow"

# Use the token for authentication
os.environ["DAGSHUB_TOKEN"] = "5b2fc63cf75726c65814691131d812b2c2db11d2"

# Initialize DagsHub
dagshub.init(repo_owner='adithyab22', repo_name='chest_cancer_classification', mlflow=True)

# Start an MLflow run and log parameters and metrics
with mlflow.start_run():
    mlflow.log_param('parameter name', 'value')
    mlflow.log_metric('metric name', 1)

Accessing as adithyab22

Initialized MLflow to track repo "adithyab22/chest_cancer_classification"

Repository adithyab22/chest_cancer_classification initialized!

2024/10/12 13:54:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-zebra-718 at: https://dagshub.com/adithyab22/chest_cancer_classification.mlflow/#/experiments/0/runs/4ba81ffc9ca04538a819b02910531966.
2024/10/12 13:54:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/adithyab22/chest_cancer_classification.mlflow/#/experiments/0.


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/adithyab22/chest_cancer_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-12 13:54:25,783:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-10-12 13:54:25,784:INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-12 13:54:25,784:INFO:common:created directory at: artifacts]
Found 102 images belonging to 2 classes.


C:\Users\Adithya\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 1.0000 - loss: 1.3285e-04
[2024-10-12 13:54:35,943:INFO:common:json file saved at: scores.json]


2024/10/12 13:54:37 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/12 13:54:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/12 13:55:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2024/10/12 13:55:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-chimp-761 at: https://dagshub.com/adithyab22/chest_cancer_classification.mlflow/#/experiments/0/runs/104fe2217c1141e8bf44f68a6a35bf74.
2024/10/12 13:55:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/adithyab22/chest_cancer_classification.mlflow/#/experiments/0.
